In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType

In [2]:
project_number = 294601891609
project_id = "crypto-busting-374123"
location = "europe-central2"
subscription_id = "bda-coinbase-topic-sub"
topic_id = "bda-coinbase-topic"
timeout = 5.0

In [3]:
# spark = SparkSession.builder.appName("Read Pub/Sub Stream").master("yarn").getOrCreate()

In [4]:
# spark.sql('add jar file:////home/bda_reddit/dependancies/spark-streaming-pubsub_2.11.jar')

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
23/01/14 01:18:13 WARN org.apache.hadoop.hive.ql.session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


DataFrame[result: int]

In [5]:
# sdf = (
#     spark.readStream.format("pubsub")
#     .option(
#         "pubsub.subscription",
#         f"projects/{project_number}/locations/{location}/subscriptions/{subscription_id}",
#     )
#     .load()
# )

In [4]:
from concurrent.futures import TimeoutError
from google.cloud import pubsub_v1

In [5]:
subscriber = pubsub_v1.SubscriberClient()

In [6]:
subscription_path = subscriber.subscription_path(project_id, subscription_id)

In [7]:
messages = []

In [8]:
def callback(message: pubsub_v1.subscriber.message.Message) -> None:
    print(f"Received {message}.")
    message.ack()
    messages.append(message.data)

In [9]:
streaming_pull_future = subscriber.subscribe(subscription_path, callback=callback)
print(f"Listening for messages on {subscription_path}..\n")

Listening for messages on projects/crypto-busting-374123/subscriptions/bda-coinbase-topic-sub..



In [ ]:
with subscriber:
    try:
        # When `timeout` is not set, result() will block indefinitely,
        # unless an exception is encountered first.
        streaming_pull_future.result(timeout=timeout)
    except TimeoutError:
        streaming_pull_future.cancel()  # Trigger the shutdown.
        streaming_pull_future.result()  # Block until the shutdown is complete.

In [13]:
import ast

In [12]:
for mes in messages:
    mes.decode('UTF-8')

In [15]:
response = [ast.literal_eval(mes.decode("UTF-8")) for mes in messages]

In [17]:
response[0]

{'side': '',
 'price': '21061.88',
 'product_id': 'BTC-USD',
 'time': '2023-01-14T23:30:03.483759Z'}

In [22]:
import pandas as pd
df = pd.DataFrame.from_dict(response)
df.tail()

,side,price,product_id,time
185,,1521.1,ETH-USD,2023-01-15T01:10:02.468969Z
186,,20683.52,BTC-USD,2023-01-15T01:10:02.026102Z
187,,0.3453,ADA-USD,2023-01-15T01:10:00.847999Z
188,,0.08478,DOGE-USD,2023-01-15T01:10:02.298163Z
189,,22.57,SOL-USD,2023-01-15T01:10:02.726285Z
